In [1]:
import pandas as pd
import numpy as np

trips = pd.read_csv('most_popular_trips.csv')
print(trips.shape)
trips.head()

(250000, 10)


,Unnamed: 0,origin_code,destination_code,n_trips,origin_state_county,destination_state_county,origin,destination,origin_state_code,destination_state_code
0,695469,6073,6037,9098075.0,"('6', '073')","('6', '037')",San Diego County,Los Angeles County,6,6
1,638859,6037,6073,9098075.0,"('6', '037')","('6', '073')",Los Angeles County,San Diego County,6,6
2,619983,6025,6073,4337119.0,"('6', '025')","('6', '073')",Imperial County,San Diego County,6,6
3,695463,6073,6025,4337119.0,"('6', '073')","('6', '025')",San Diego County,Imperial County,6,6
4,682885,6065,6037,4009696.0,"('6', '065')","('6', '037')",Riverside County,Los Angeles County,6,6


In [2]:
state_fips = pd.read_csv('StatesFIPSCodes.csv')
state_fips.head(2)

,STATE_FIPS,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533


In [3]:
# Are there any counties of same name but different state?
trips['destination'].nunique()

1869

In [4]:
trips['destination_code'].nunique()

3145

In [5]:
pd.Series([str(tup) for tup in trips['destination_state_county']]).nunique()

3145

In [6]:
# The County Name "Jefferson" is Present in 25 Different States
trips[trips['destination'] == 'Jefferson County']['destination_state_code'].nunique()

25

In [7]:
def get_state_name(state_code):
    """Returns state name when given State FIPS Code"""
    query_string = f"{state_code} == `STATE_FIPS`"
    return state_fips.query(query_string)['STATE_NAME'].iloc[0]

In [8]:
%%time
# For Destination
state_name = []
for i in trips['destination_state_code']:
    state_name.append(get_state_name(i))

CPU times: user 4min 7s, sys: 2.16 s, total: 4min 9s
Wall time: 4min 8s


In [9]:
trips.insert(10, 'dest_state', state_name)

In [10]:
%%time
# For Origin
state_name = []
for i in trips['origin_state_code']:
    state_name.append(get_state_name(i))

CPU times: user 4min 12s, sys: 1.93 s, total: 4min 13s
Wall time: 4min 12s


In [11]:
trips.insert(11, 'orig_state', state_name)

In [12]:
trips.head()

,Unnamed: 0,origin_code,destination_code,n_trips,origin_state_county,destination_state_county,origin,destination,origin_state_code,destination_state_code,dest_state,orig_state
0,695469,6073,6037,9098075.0,"('6', '073')","('6', '037')",San Diego County,Los Angeles County,6,6,California,California
1,638859,6037,6073,9098075.0,"('6', '037')","('6', '073')",Los Angeles County,San Diego County,6,6,California,California
2,619983,6025,6073,4337119.0,"('6', '025')","('6', '073')",Imperial County,San Diego County,6,6,California,California
3,695463,6073,6025,4337119.0,"('6', '073')","('6', '025')",San Diego County,Imperial County,6,6,California,California
4,682885,6065,6037,4009696.0,"('6', '065')","('6', '037')",Riverside County,Los Angeles County,6,6,California,California


In [13]:
def county_state(county, state):
    county_state = str(county) + str(state)

In [14]:
# Destination
dest_full = []
for i, row in trips.iterrows():
    dest_full.append(str(row[7]) + ', ' + str(row[10]))
trips.insert(12, 'dest_full', dest_full)

In [15]:
# Origin
orig_full = []
for i, row in trips.iterrows():
    orig_full.append(str(row[6]) + ', ' + str(row[11]))

trips.insert(13, 'orig_full', orig_full)

In [19]:
# rename unnamed column
trips['original_id'] = trips['Unnamed: 0']
trips = trips.drop('Unnamed: 0', axis=1)

In [20]:
trips.head()

,origin_code,destination_code,n_trips,origin_state_county,destination_state_county,origin,destination,origin_state_code,destination_state_code,dest_state,orig_state,dest_full,orig_full,original_id
0,6073,6037,9098075.0,"('6', '073')","('6', '037')",San Diego County,Los Angeles County,6,6,California,California,"Los Angeles County, California","San Diego County, California",695469
1,6037,6073,9098075.0,"('6', '037')","('6', '073')",Los Angeles County,San Diego County,6,6,California,California,"San Diego County, California","Los Angeles County, California",638859
2,6025,6073,4337119.0,"('6', '025')","('6', '073')",Imperial County,San Diego County,6,6,California,California,"San Diego County, California","Imperial County, California",619983
3,6073,6025,4337119.0,"('6', '073')","('6', '025')",San Diego County,Imperial County,6,6,California,California,"Imperial County, California","San Diego County, California",695463
4,6065,6037,4009696.0,"('6', '065')","('6', '037')",Riverside County,Los Angeles County,6,6,California,California,"Los Angeles County, California","Riverside County, California",682885


In [34]:
# TODO: Why are these not equal?
print(trips['destination_code'].nunique())
print(trips['dest_full'].nunique())

3145
3139


In [35]:
# Save Data
trips.to_csv('orig_dest_detailed.csv')